# TODO

- [x] IDENTIFICAÇÃO DAS CIDADES LITORÂNEAS  
- [x] LEVANTAMENTO DA QUANTIDADE DE HOSPITAIS  
- [ ] LEVANTAMENTO DA SEGURANÇA  
- [ ] LEVANTAMENTO DA MÉDIA DE ALUGUÉIS  
- [ ] LEVANTAMENTO DOS PREÇOS DE ALIMENTO

# IMPORTAÇÃO DE BIBLIOTECAS

In [2]:
import polars as pl
import os
import duckdb

In [3]:
AUXILIAR_PATH = r'finding-home/finding_home/auxiliar/'

# IMPORTAÇÃO DE DADOS

## CIDADES

In [4]:
from unicodedata import normalize 
def custnorm(In_series):
    for i, x in enumerate(In_series):
        newvalue = normalize('NFKD',x).encode('ascii', errors='ignore').decode('utf-8')
        if newvalue != x:
            In_series[i]=newvalue
    return In_series

In [5]:
cities = pl.read_csv(os.path.join(AUXILIAR_PATH, 'CIDADES_LITORANEAS_IBGE.csv'), separator=';') \
           .filter(pl.col('NM_REGIAO').is_in(['Sul', 'Sudeste', 'Nordeste'])) \
           .with_columns(pl.col('NM_MUN').str.to_uppercase().alias('MUNICIPIO')) \
           .with_columns(pl.col('MUNICIPIO').map(custnorm).alias('MUNICIPIO'))

/tmp/ipykernel_12575/3017128636.py:4: DeprecationWarning: `map` is deprecated. It has been renamed to `map_batches`.
  .with_columns(pl.col('MUNICIPIO').map(custnorm).alias('MUNICIPIO'))


## HOSPITAIS

In [6]:
hospitals = pl.read_csv(os.path.join(AUXILIAR_PATH, 'hospitais.csv'), separator='|', encoding='cp1252', truncate_ragged_lines=True)

In [7]:
hospitals_stat = duckdb.sql("""
                            SELECT 
                                    REGIAO
                                ,   UF
                                ,   MUNICIPIO
                                ,   DESC_NATUREZA_JURIDICA
                                ,   SUM(CASE WHEN DS_TIPO_UNIDADE == 'HOSPITAL GERAL' THEN 1 ELSE 0 END) AS HOSPITAL_GERAL
                                ,   SUM(CASE WHEN DS_TIPO_UNIDADE == 'PRONTO SOCORRO GERAL' THEN 1 ELSE 0 END) AS PRONTO_SOCORRO_GERAL 
                                ,   SUM(CASE WHEN DS_TIPO_UNIDADE == 'UNIDADE MISTA' THEN 1 ELSE 0 END) AS UNIDADE_MISTA
                            
                            FROM hospitals
                            GROUP BY REGIAO
                                ,   UF
                                ,   MUNICIPIO
                                ,   DESC_NATUREZA_JURIDICA

                            ORDER BY REGIAO
                                ,   UF
                                ,   MUNICIPIO
                                ,   DESC_NATUREZA_JURIDICA
                            
                            """).to_df()

In [8]:
hospitals_stat.head()

,REGIAO,UF,MUNICIPIO,DESC_NATUREZA_JURIDICA,HOSPITAL_GERAL,PRONTO_SOCORRO_GERAL,UNIDADE_MISTA
0,CENTRO-OESTE,DF,BRASILIA,HOSPITAL_FILANTRÓPICO,4.0,0.0,0.0
1,CENTRO-OESTE,DF,BRASILIA,HOSPITAL_PRIVADO,26.0,0.0,0.0
2,CENTRO-OESTE,DF,BRASILIA,HOSPITAL_PÚBLICO,16.0,0.0,0.0
3,CENTRO-OESTE,GO,ABADIANIA,HOSPITAL_PÚBLICO,0.0,0.0,1.0
4,CENTRO-OESTE,GO,ACREUNA,HOSPITAL_PRIVADO,2.0,0.0,0.0


# ALIMENTOS

In [62]:
food = pl.read_csv(os.path.join(AUXILIAR_PATH, 'ALIMENTOS_CONAB.csv'), separator=';').melt(id_vars=['UF', 'PRODUTO'], value_vars=pl.selectors.numeric(), variable_name='MES_REF', value_name='PRECO')

In [63]:
food.columns

['UF', 'PRODUTO', 'MES_REF', 'PRECO']

In [67]:
food

UF,PRODUTO,MES_REF,PRECO
str,str,str,f64
"""ES""","""AÇÚCAR CRISTAL…","""01/01/2023""",7.96
"""GO""","""AÇÚCAR CRISTAL…","""01/01/2023""",7.81
"""PR""","""AÇÚCAR CRISTAL…","""01/01/2023""",7.61
"""RO""","""AÇÚCAR CRISTAL…","""01/01/2023""",7.2
"""TO""","""AÇÚCAR CRISTAL…","""01/01/2023""",7.99
"""DF""","""AÇÚCAR CRISTAL…","""01/01/2023""",16.91
"""ES""","""AÇÚCAR CRISTAL…","""01/01/2023""",18.32
"""GO""","""AÇÚCAR CRISTAL…","""01/01/2023""",17.39
"""MA""","""AÇÚCAR CRISTAL…","""01/01/2023""",19.1


In [84]:
foods = duckdb.sql("""
   PIVOT food ON PRODUTO USING AVG(PRECO) AS PRECO GROUP BY UF
""").to_df()

In [87]:
foods.fillna(0, inplace=True)

In [89]:
foods.columns

Index(['UF', 'ARROZ BENEFICIADO VERMELHO (kg)_PRECO',
       'ARROZ LONGO FINO BENEFICIADO TIPO 1 (2 kg)_PRECO',
       'ARROZ LONGO FINO BENEFICIADO TIPO 1 (5 kg)_PRECO',
       'ARROZ LONGO FINO BENEFICIADO TIPO 1 (5.00 kg)_PRECO',
       'ARROZ LONGO FINO BENEFICIADO TIPO 1 (kg)_PRECO',
       'ARROZ LONGO FINO BENEFICIADO TIPO 2 (kg)_PRECO',
       'ARROZ LONGO FINO PARBOILIZADO TIPO 1 (kg)_PRECO',
       'AÇÚCAR CRISTAL (2 kg)_PRECO', 'AÇÚCAR CRISTAL (5 kg)_PRECO',
       'AÇÚCAR CRISTAL (5.00 kg)_PRECO', 'AÇÚCAR CRISTAL (kg)_PRECO',
       'AÇÚCAR REFINADO (kg)_PRECO', 'BATATA INGLESA (kg)_PRECO',
       'BATATA INGLESA LISA (kg)_PRECO', 'CAFÉ MOÍDO E TORRADO (250 g)_PRECO',
       'CAFÉ MOÍDO E TORRADO (500 g)_PRECO', 'CARNE BOVINA ACÉM (kg)_PRECO',
       'CARNE BOVINA CHARQUE PA CRAY-O-VAC (500 g)_PRECO',
       'CARNE BOVINA CHARQUE PA MANTA (kg)_PRECO',
       'CARNE BOVINA COSTELA (kg)_PRECO',
       'CARNE BOVINA COSTELA MINDINHA (kg)_PRECO',
       'CARNE BOVINA DIANTEIRO

# UNIÃO CIDADES & HOSPITAIS

In [9]:
city_hosp = duckdb.sql("""
           SELECT 
              CT.CD_MUN
            , CT.NM_MUN
            , CT.AREA_KM2
            , CT.CD_UF
            , CT.NM_UF
            , HS.*

           FROM hospitals_stat AS HS

           JOIN cities AS CT
           ON CT.SIGLA_UF = HS.UF
           AND CT.MUNICIPIO = HS.MUNICIPIO
           """).to_df()

In [11]:
city_hosp.head()

,CD_MUN,NM_MUN,AREA_KM2,CD_UF,NM_UF,REGIAO,UF,MUNICIPIO,DESC_NATUREZA_JURIDICA,HOSPITAL_GERAL,PRONTO_SOCORRO_GERAL,UNIDADE_MISTA
0,2702306,Coruripe,897.800,27,Alagoas,NORDESTE,AL,CORURIPE,HOSPITAL_PRIVADO,1.0,0.0,0.0
1,2704302,Maceió,509.320,27,Alagoas,NORDESTE,AL,MACEIO,HOSPITAL_FILANTRÓPICO,4.0,0.0,0.0
2,2704302,Maceió,509.320,27,Alagoas,NORDESTE,AL,MACEIO,HOSPITAL_PRIVADO,12.0,0.0,0.0
3,2704302,Maceió,509.320,27,Alagoas,NORDESTE,AL,MACEIO,HOSPITAL_PÚBLICO,4.0,0.0,0.0
4,2900801,Alcobaça,1477.929,29,Bahia,NORDESTE,BA,ALCOBACA,HOSPITAL_PÚBLICO,1.0,0.0,0.0


In [13]:
duckdb.sql(""" SELECT DISTINCT(MUNICIPIO) FROM city_hosp where UF == 'SP' LIMIT 10""")

┌───────────────┐
│   MUNICIPIO   │
│    varchar    │
├───────────────┤
│ BERTIOGA      │
│ IGUAPE        │
│ SAO VICENTE   │
│ GUARUJA       │
│ SANTOS        │
│ CARAGUATATUBA │
│ PRAIA GRANDE  │
│ ILHABELA      │
│ PERUIBE       │
│ SAO SEBASTIAO │
├───────────────┤
│    10 rows    │
└───────────────┘

# LINKS AUXILIARES

- [API IBGE CIDADES](https://servicodados.ibge.gov.br/api/v3/agregados/6579/periodos/2021/variaveis/9324?localidades=N6[1100015])
- [API IBGE AGREGADA](https://servicodados.ibge.gov.br/api/docs/agregados?versao=3)
- [ALIMENTOS CONAB](https://sisdep.conab.gov.br/precosiagroweb/)
